## Converting NWB 1.0 files to NWB 1.2

Tested with Python 2.7.11

In [1]:
from pynwb import NWBFile
from datetime import datetime

/Users/xiaoxuanj/anaconda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [75]:
import h5py as h5
import numpy as np

In [6]:
nwb1_file = 'mouse366119_probe_surf.nwb'
nwb1 = h5.File(nwb1_file)

AttributeError: 'module' object has no attribute 'version'

In [17]:
nwb1['general']['experimenter'].value

'experimenter'

In [1]:
import json
fname = '/Users/Josh/Data/mouse366119/probeE/experiment2/recording2/structure.oebin'

In [3]:
info = json.load(open(fname))

In [14]:
info['continuous'][0]['sample_rate']

30000

In [35]:
nwb1['general']['subject']['subject_id'].value

'subject_id'

In [119]:
nwb2 = NWBFile('NWB 1.0 > NWB 1.2 converter',        # source
                  nwb1['session_description'].value, # session description
                  nwb1['identifier'].value,          # identifier
                  nwb1['session_start_time'].value,  # date
                  experimenter = nwb1['general']['experimenter'].value,
                  lab = nwb1['general']['lab'].value,
                  institution = nwb1['general']['lab'].value,
                  experiment_description = nwb1['general']['experiment_description'].value,
                  session_id='SESSION_ID')

In [82]:
nwb2.nwb_version

'1.2.0'

In [120]:
device = nwb2.create_device(name='Neuropixels Prototype Rig', source="a source")

In [71]:
probes = [p for p in nwb1['processing'].keys() if p[:5] == 'probe']

In [72]:
probes

[u'probeA', u'probeB', u'probeC', u'probeD', u'probeE', u'probeF']

In [73]:
locations = {'probeA' : 'AM',
             'probeB' : 'PM',
             'probeC' : 'V1',
             'probeD' : 'LM',
             'probeE' : 'AL',
             'probeF' : 'RL'}

xlocs = [16, 48, 0, 32]

In [121]:
for probe in probes:

    electrode_name = probe
    source = "Neuropixels Probe"
    description = "Phase 3a"
    location = locations[probe]

    electrode_group = nwb2.create_electrode_group(electrode_name,
                                                 source=source,
                                                 description=description,
                                                 location=location,
                                                 device=device)
    
    for idx in range(0,384):
        nwb2.add_electrode(idx,
                          x=float(xlocs[idx % 4]), y=float(idx/2*20), z=np.nan,
                          imp=np.nan,
                          location='unknown', filtering='300 Hz highpass',
                          description='channel %s' % idx, group=electrode_group)

In [118]:
del nwb2

In [122]:
from pynwb.ecephys import Clustering

module = nwb2.create_processing_module('ecephys', # name
                                       'ecephys', #source
                                        'Clustering module' # description
                                      )

for probe in probes:
    
    clustering = Clustering(probe, # source
                                   'Kilosort outputs', # description
                                   nwb1['processing'][probe]['Clustering']['num'].value, # cluster ID
                                   nwb1['processing'][probe]['Clustering']['peak_over_rms'].value, #peak over RMS
                                   nwb1['processing'][probe]['Clustering']['times'].value, # times
                                   probe + '_clustering') # name
    
    module.add_data_interface(clustering)                               


In [90]:
nwb1['processing']['probeA']['Clustering'].keys()

[u'cluster_nums', u'num', u'peak_over_rms', u'times']

In [137]:
nwb2.modules['ecephys'].containers[0].times

array([   73.07599952,    76.09473792,    76.3832384 , ...,  4866.47181501,
        4870.83655567,  4888.74231907])

In [117]:
module.containers

(<pynwb.ecephys.Clustering at 0x10a07ced0>,
 <pynwb.ecephys.Clustering at 0x10a01c650>)

In [175]:
nwb1['stimulus']['presentation']['flash_250ms_1']['timestamps'].value[:,0]

array([ 1392.71284,  1394.71397,  1396.71513,  1398.71745,  1400.71927,
        1402.72063,  1404.72067,  1406.72426,  1408.72514,  1410.72776,
        1412.72848,  1414.73106,  1416.7323 ,  1418.73349,  1420.73517,
        1422.73627,  1424.73767,  1426.73944,  1428.74182,  1430.74416,
        1432.74565,  1434.74783,  1436.74938,  1438.75117,  1440.75262,
        1442.75366,  1444.75592,  1446.75701,  1448.75875,  1450.76054,
        1452.76286,  1454.76367,  1456.76614,  1458.76803,  1460.76905,
        1462.76986,  1464.77264,  1466.77313,  1468.77473,  1470.77594,
        1472.77921,  1474.78063,  1476.78326,  1478.78413,  1480.7864 ,
        1482.78791,  1484.78983,  1486.79067,  1488.79257,  1490.79489,
        1492.79646,  1494.79823,  1496.79923,  1498.80087,  1500.80345,
        1502.80511,  1504.80658,  1506.80845,  1508.80992,  1510.81093,
        1512.81318,  1514.81525,  1516.81606,  1518.81852,  1520.81999,
        1522.82111,  1524.82267,  1526.82446,  1528.82675,  1530

In [194]:
nwb1['stimulus']['presentation']['perceptual_grouping02_2']['feature_units']

<HDF5 dataset "feature_units": shape (6,), type "|S9">

In [195]:
from pynwb.misc import AbstractFeatureSeries

module = nwb2.create_processing_module('stimulus', # name
                                       'stimulus', #source
                                        'stimulus info' # description
                                      )

for stimulus in nwb1['stimulus']['presentation'].keys():
    
    afs = AbstractFeatureSeries(stimulus, # name
                          stimulus, # source
                          nwb1['stimulus']['presentation'][stimulus]['feature_units'].value, # units
                          nwb1['stimulus']['presentation'][stimulus]['features'].value, # features
                          data = nwb1['stimulus']['presentation'][stimulus]['data'].value, # data
                          timestamps = nwb1['stimulus']['presentation'][stimulus]['timestamps'].value)

    module.add_data_interface(afs)